**Drive Connection**

In [1]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive


/
Mounted at /content/gdrive
 Ayvos-Veri   detectron2-hand   Staj	      yolov4-tiny-deprem
 Classroom   'My Drive'         yolov4-tiny


**Install Library**

In [2]:
!pip install pyyaml==5.1
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 274 kB 36.9 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=91bfe6d8d4594cdb72440c0b9dadb1c5c5fb9b6930d80214d61bf8b3544181b8
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.5 MB 15 kB/s 
     |████████████████████████████████| 17.3 MB 37.7 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113

In [3]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# After this step it will ask you to restart the runtime, please do it.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
     |████████████████████████████████| 6.3 MB 540 kB/s 
     |████████████████████████████████| 50 kB 5.7 MB/s 
     |████████████████████████████████| 151 kB 51.1 MB/s 
     |████████████████████████████████| 79 kB 8.5 MB/s 
     |████████████████████████████████| 130 kB 62.8 MB/s 
     |████████████████████████████████| 843 kB 56.0 MB/s 
     |████████████████████████████████| 117 kB 65.3 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=ec02e4aa38b193de8142abfe82c02c67578c1cadc25b39c6924e9d8498214f7e
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=c719a3ff1b5de6490f7

**Import Libraries**

In [1]:
import torch
assert torch.__version__.startswith("1.8") 
import torchvision
import cv2
import uuid


In [2]:
import os
import numpy as np
import json
import random
import matplotlib.pyplot as plt
%matplotlib inline

from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog

In [3]:
def get_dicts(directory, classes):
    print(os.listdir(directory))
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        
        record["file_name"] = filename
        record["height"] = 480
        record["width"] = 848
      
        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            px = [a[0] for a in anno['points']] # x coord
            py = [a[1] for a in anno['points']] # y-coord
            poly = [(x, y) for x, y in zip(px, py)] # poly for segmentation
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": classes.index(anno['label']),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [4]:
classes = ['dolu', 'bos_el']

data_path = '/mydrive/detectron2-hand/data/'

for i in ["train", "test"]:
    DatasetCatalog.register(
        "category_" + i, 
        lambda i=i: get_dicts(data_path+i, classes)
    )
    MetadataCatalog.get("category_" + i).set(thing_classes=classes)

microcontroller_metadata = MetadataCatalog.get("category_train")

**Training the Detectron2**

In [5]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer

In [7]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("category_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)

[08/17 20:11:36 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


In [ ]:
trainer.train()

[08/17 20:11:45 d2.engine.train_loop]: Starting training from iteration 0
[08/17 20:11:57 d2.utils.events]:  eta: 0:09:11  iter: 19  total_loss: 1.775  loss_cls: 0.9418  loss_box_reg: 0.1134  loss_mask: 0.6923  loss_rpn_cls: 0.01432  loss_rpn_loc: 0.007593  time: 0.5470  data_time: 0.0189  lr: 4.9953e-06  max_mem: 2675M
[08/17 20:12:08 d2.utils.events]:  eta: 0:08:55  iter: 39  total_loss: 1.694  loss_cls: 0.8631  loss_box_reg: 0.0998  loss_mask: 0.6898  loss_rpn_cls: 0.03426  loss_rpn_loc: 0.007686  time: 0.5432  data_time: 0.0075  lr: 9.9902e-06  max_mem: 2675M
[08/17 20:12:19 d2.utils.events]:  eta: 0:08:48  iter: 59  total_loss: 1.512  loss_cls: 0.6656  loss_box_reg: 0.1282  loss_mask: 0.6829  loss_rpn_cls: 0.02039  loss_rpn_loc: 0.008538  time: 0.5538  data_time: 0.0070  lr: 1.4985e-05  max_mem: 2675M
[08/17 20:12:30 d2.utils.events]:  eta: 0:08:38  iter: 79  total_loss: 1.284  loss_cls: 0.4701  loss_box_reg: 0.1055  loss_mask: 0.6748  loss_rpn_cls: 0.01636  loss_rpn_loc: 0.006554

**Using the Trained Model**

**Model Copy**

In [ ]:
!cp -r ./output /mydrive/detectron2-hand/

**Get Model pth file**

In [8]:
cfg.MODEL.WEIGHTS = os.path.join("/mydrive/detectron2-hand/output", "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 
cfg.DATASETS.TEST = ("skin_test", )
predictor = DefaultPredictor(cfg)

In [ ]:
import glob
jpg_list=glob.glob("/mydrive/detectron2-hand/data/test/*.jpg")
counter=0
for i in range(len(jpg_list)):  
    img = cv2.imread(jpg_list[counter])
    outputs = predictor(img)
    v = Visualizer(img[:, :, ::-1],
                   metadata=microcontroller_metadata, 
                   scale=0.8, 
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

    unique_filename=str(uuid.uuid1())
    cv2.imwrite("/mydrive/detectron2-hand/results/"+str(counter)+ "-" + str(unique_filename)[:8] +".jpg", v.get_image()[:, :, ::-1])
    counter+=1
    